In [1]:
import gymnasium as gym
import numpy as np

In [2]:
desc=["SFFF", "FHHH", "FFFF", "HFHF", "FFGF"]

env = gym.make('FrozenLake-v1', desc=desc, map_name="5x5", is_slippery=False, render_mode="human") 
observation, info = env.reset()

In [ ]:
# Define parameters
gamma = 0.9  # Discount factor
alpha = 0.1  # Learning rate
epsilon = 0.05  # Epsilon-greedy parameter
num_episodes = 1000

# Initialize Q-values
num_states = env.observation_space.n
num_actions = env.action_space.n
Q = np.zeros((num_states, num_actions))

# Q-learning algorithm
for episode in range(num_episodes):
    state = env.reset()[0]
    done = False
    
    while not done:
        # Epsilon-greedy action selection
        if np.random.rand() < epsilon:
            action = env.action_space.sample()  # Random action
        else:
            ind = 0
            maxInd = -1
            maxNum = -1
            allZeros = 0
            for num in Q[state]:
                if num > maxNum:
                    maxNum = num
                    maxInd = ind
                if num == 0:
                    allZeros+=1
                ind += 1
                
            if allZeros == 4 or maxInd== -1:
                maxInd = env.action_space.sample()
            action = maxInd  # Greedy action
        
        # Take action and observe next state and reward
        step = env.step(action)
        next_state = step[0]
        reward = step[1]
        done = step[2]
        
        if reward == 0 and done:
            print("hit lake :C")
            reward = -1
        
        # Update Q-value
        Q[state, action] += alpha * (reward + gamma * np.max(Q[next_state, :]) - Q[state, action])
        
        if reward == 1:
            print("Hit gift!")
            print(Q)
        
        # Move to next state
        state = next_state

# Optimal policy
optimal_policy = np.argmax(Q, axis=1)

print("Optimal policy:")
print(optimal_policy)


hit lake :C
hit lake :C
hit lake :C
hit lake :C
hit lake :C
hit lake :C
hit lake :C
hit lake :C
hit lake :C
hit lake :C
hit lake :C
